**teneva_research: power**

Исследование поэлементного возведения TT-тензоров в степень

---

# Установка и импорт модулей

In [1]:
import numpy as np
import teneva # Используется версия 0.11.4!
from time import perf_counter as tpc

# Подготовка тензоров

Мы рассматриваем только те модельные функции, для которых известна явная форма TT-ядер и которые имеют минимум в нуле (поскольку мы берем нечетное число узлов сетки, то мульти-индекс тензора попадет точно в оптимум):

In [2]:
func_names = ['Alpine', 'Rastrigin', 'Rosenbrock', 'Schwefel']
           # 'Exponential' - min и max перепутаны, пока отложил
           # 'Grienwank' - плохо работает ;(
d          = 100       # Размерность функции
n          = 2**6 + 1  # Число узлов сетки
data       = []

Для каждой функции мы:
- Явно строим ее TT-ядра
- Находим максимальное по модулю значение $y_{max}$, используя "optima_tt"
- Вычитаем из TT-тензора $y_{max}$ без округления (теперь в начале координат функция имеет минимум, который одновременно является максимальным по модулю значением)
- Сохраняем значение полученного TT-тензора в начале координат как референсное

In [3]:
for func in teneva.func_demo_all(d, names=func_names):
    func.set_grid(n, kind='uni')
    
    t = tpc()
    func.cores()
    t = tpc() - t
    
    y_max = teneva.optima_tt_max(func.Y)[-1]
    func.Y = teneva.sub(func.Y, y_max)
    
    i = teneva.poi_to_ind(func.x_min, func.a, func.b, func.n)
    y = teneva.get(func.Y, i)
    data.append({'name': func.name, 'Y': func.Y, 'i': i, 'y': y})

    text = ''
    text += func.name + ' '*max(0, 12-len(func.name)) + '| '
    text += f'r = {teneva.erank(func.Y):-6.2f} | '
    text += f'y = {y:-24.7f} | '
    text += f't = {t:-8.3f} | '
    print(text)

Alpine      | r =   3.00 | y =             -864.0927477 | t =    2.611 | 
Rastrigin   | r =   3.00 | y =            -3999.1547013 | t =    2.598 | 
Rosenbrock  | r =   4.00 | y =          -386680.6600335 | t =    3.304 | 
Schwefel    | r =   3.00 | y =           -83775.8442853 | t =    2.389 | 


# Явное возведение в степень

Будем возводить тензор в степень s, округляя результат после каждой операции и дополнительно нормируя тензор (мы сохраняем все промежуточные нормировки и используем их при вычислении итогового результата), также найдем индекс оптимума (максимума по модулю) для тензора, возведенного в степень, и оценим ошибку (берем значение исходного тензора в найденном мульти-индексе и смотрим на отклонение от точного значения).

In [4]:
def power_mul(Y, s, e_trunc):
    Z = teneva.copy(Y)
    p = 1.
    for _ in range(s-1):
        Z = teneva.mul(Z, Y)
        Z = teneva.truncate(Z, e_trunc)
        p_curr = teneva.norm(Z)
        Z = teneva.mul(Z, 1./p_curr)
        Z = teneva.truncate(Z, e_trunc)
        p *= p_curr
    return Z, p

In [6]:
def run(data, s, power, e_trunc=1.E-12, name='MUL'):
    print('-'*103 + f'\n-> RUN {name}  | d = {d:-3d}, n = {n:-5d}, power = {s}, e_trunc = {e_trunc:-7.1e}\n' + '-'*103)
    
    for item in data:
        t = tpc()
        Z, p = power(item['Y'], s, e_trunc)
        t = tpc() - t

        z = teneva.get(Z, item['i'])
        z = (z * p)**(1./s)
        
        e_val = abs(abs(item['y']) - abs(z))

        i_opt = teneva.optima_tt_max(Z, k=10)[0]
        y_opt = teneva.get(item['Y'], i_opt)
        e_opt = abs(item['y'] - y_opt)

        text = ''
        text += item['name'] + ' '*max(0, 12-len(item['name'])) + '| '
        text += f'e val = {e_val:7.1e} | '
        text += f'e opt = {e_opt:7.1e} | '
        #text += f'r full = {teneva.erank(item["Y"])**s:-10.1f} | '
        text += f'r real = {teneva.erank(Z):-5.1f} | '
        text += f't = {t:-8.3f} | '
        print(text)

In [7]:
run(data, 8, power_mul)

-------------------------------------------------------------------------------------------------------
-> RUN MUL  | d = 100, n =    65, power = 8, e_trunc = 1.0e-12
-------------------------------------------------------------------------------------------------------
Alpine      | e val = 7.6e-04 | e opt = 0.0e+00 | r real =   5.8 | t =    0.695 | 
Rastrigin   | e val = 5.0e-02 | e opt = 0.0e+00 | r real =   5.9 | t =    0.740 | 
Rosenbrock  | e val = 8.8e-03 | e opt = 8.3e+01 | r real =  18.1 | t =   12.167 | 
Schwefel    | e val = 4.1e+00 | e opt = 0.0e+00 | r real =   5.9 | t =    0.715 | 


# Возведение сразу в большую степень с использованием `cross_act`

In [8]:
def power_act2_exp(Y, s, e_trunc, act_e=1.E-8, act_nswp=5, act_r=1, act_dr=2, act_dr2=0):
    def f(X):
        return np.exp(s*X[:, 0])
        #return X.reshape(-1)
    R = teneva.tensor_rand(teneva.shape(Y), r=act_r)
    Z = teneva.cross_act(f, [Y], R,
        e=act_e, nswp=act_nswp, r=9999, dr=act_dr, dr2=act_dr2, log=False)
    Z = teneva.truncate(Z, e=e_trunc)
    return Z, 1.

In [9]:
Z = run(data, 0.1, power_act2_exp)

-------------------------------------------------------------------------------------------------------
-> RUN MUL  | d = 100, n =    65, power = 0.1, e_trunc = 1.0e-12
-------------------------------------------------------------------------------------------------------
Alpine      | e val = 8.6e+02 | e opt = 8.6e+02 | r real =   1.0 | t =    0.395 | 
Rastrigin   | e val = 4.0e+03 | e opt = 4.0e+03 | r real =   1.0 | t =    0.372 | 


/Users/andrei/opt/anaconda3/lib/python3.8/site-packages/teneva-0.11.4-py3.8.egg/teneva/core/act_two.py:35: RuntimeWarning: divide by zero encountered in double_scalars
/Users/andrei/opt/anaconda3/lib/python3.8/site-packages/teneva-0.11.4-py3.8.egg/teneva/core/act_two.py:35: RuntimeWarning: invalid value encountered in double_scalars


Rosenbrock  | e val = 3.9e+05 | e opt = 3.9e+05 | r real =   1.0 | t =    0.375 | 
Schwefel    | e val = 8.4e+04 | e opt = 3.4e+04 | r real =   1.0 | t =    0.404 | 


/Users/andrei/opt/anaconda3/lib/python3.8/site-packages/teneva-0.11.4-py3.8.egg/teneva/core/optima.py:154: RuntimeWarning: invalid value encountered in true_divide


In [116]:
import scipy as sp


from teneva import copy
from teneva import matrix_svd
from teneva import _reshape

In [117]:
def core_stab(G, p0=0, thr=1.E-100):
    """Scaling for the passed TT-core, i.e., G -> (Q, p), G = 2^p * Q.

    Args:
        G (np.ndarray): TT-core in the form of 3-dimensional array.
        p0 (int): optional initial value of the power-factor (it will be added
            to returned value "p").
        thr (float): threshold value for applying scaling (if the maximum
            modulo element in the TT-core is less than this value, then scaling
            will not be performed).

    Returns:
        (np.ndarray, int): scaled TT-core (Q) and power-factor (p), such that
            G = 2^p * Q.

    """
    v_max = float(np.max(np.abs(G)))

    if v_max <= thr:
        return G, p0

    p = int(np.floor(np.log2(v_max)))
    Q = G / 2**p

    return Q, p0 + p

In [118]:
def orthogonalize(Y, k=None, use_stab=False):
    """Orthogonalize TT-tensor.

    Args:
        Y (list): TT-tensor.
        k (int): the leading mode for orthogonalization. The TT-cores 0, 1, ...,
            k-1 will be left-orthogonalized and the TT-cores k+1, k+2, ..., d-1
            will be right-orthogonalized. It will be the last mode by default.
        use_stab (bool): if flag is set, then function will also return the
            second argument "p", which is the factor of 2-power.

    Returns:
        list: orthogonalized TT-tensor.

    """
    d = len(Y)
    k = d - 1 if k is None else k

    if k is None or k < 0 or k > d-1:
        raise ValueError('Invalid mode number')

    Z = copy(Y)
    p = 0

    for i in range(d-1, k, -1):
        orthogonalize_right(Z, i, inplace=True)
        if use_stab:
            Z[i-1], p = core_stab(Z[i-1], p)

    return (Z, p) if use_stab else Z

In [119]:
def orthogonalize_right(Y, i, inplace=False):
    """Right-orthogonalization for TT-tensor.

    Args:
        Y (list): d-dimensional TT-tensor.
        i (int): mode for orthogonalization (> 0 and <= d-1).
        inplace (bool): if flag is set, then the original TT-tensor (i.e.,
            the function argument) will be transformed. Otherwise, a copy of
            the TT-tensor will be returned.

    Returns:
        list: TT-tensor with right orthogonalized i-th mode.

    """
    d = len(Y)

    if i is None or i <= 0 or i > d-1:
        raise ValueError('Invalid mode number')

    Z = Y if inplace else copy(Y)

    r2, n2, r3 = Z[i].shape
    G2 = _reshape(Z[i], (r2, n2 * r3))
    R, Q = sp.linalg.rq(G2, mode='economic', check_finite=False)
    Z[i] = _reshape(Q, (Q.shape[0], n2, r3))

    r1, n1, r2 = Z[i-1].shape
    G1 = _reshape(Z[i-1], (r1 * n1, r2))
    G1 = G1 @ R
    Z[i-1] = _reshape(G1, (r1, n1, G1.shape[1]))

    return Z


In [120]:
Z = run(data, 0.1, power_act2_exp)

-------------------------------------------------------------------------------------------------------
-> RUN MUL  | d = 100, n =    65, power = 0.1, e_trunc = 1.0e-12
-------------------------------------------------------------------------------------------------------


In [121]:
Z, p = orthogonalize(Z, 0, use_stab=True)

In [125]:
Z[0].dtype

dtype('O')

In [9]:
G = Z[0]
r1, n, r2 = G.shape
Q = G.reshape(n, r2)
G = Z[1]
Q = np.einsum('kr,riq->kiq', Q, G, optimize='optimal')

---